# HW №1
## Task 1

In [1]:
%%capture
!pip3 install qiskit

In [2]:
import numpy as np
import qiskit as qk
from sympy import *

In [3]:
with open('./token', 'r') as token_file:
    token = token_file.read()

In [4]:
%%capture
qk.IBMQ.save_account(token, overwrite = True)
qk.IBMQ.load_account()

In [5]:
provider = qk.IBMQ.get_provider(hub = 'ibm-q')

In [6]:
devices = provider.backends(filters=lambda x: (3 <= x.configuration().n_qubits <= 5) and not x.configuration().simulator)

In [7]:
devices

[<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [ ]:
%%capture
real_backend = qk.providers.ibmq.least_busy(devices)
print(real_backend.configuration().n_qubits)

In [8]:
simd_backend = qk.Aer.get_backend('qasm_simulator')

## Task 2

Since we assume the following form of QBit wavefunction:
$\vert \psi \rangle = \sin \theta \vert 0 \rangle + \exp{i\varphi} \cos \theta \vert 1 \rangle$

Actually we measure kinda projection on $Z$ - axis that corresponds $\theta$ angle. Thus $\phi$ angle describes projection onto $X,~Y$ axes. To fix it we can rotate system (or alternavely Vector) in the following way: $X,~Y,~Z \rightarrow Z,~X,~Y$. To do this we should choose axis of rotation, obviously it is normalized vector $(1,~1,~1)$ and angle to rotate $\alpha = 2 \pi / 3$.

To make rotation itself we should compute corresponding axis. We know the form of such rotations (it can be derived from infinitesimal rotations): $\hat{R}_{\vec{n}} (\alpha)= e^{-i \vec{n} \cdot \hat{\vec{\sigma}} \alpha / 2}$.
Since all sigma's satisfy $\hat{\sigma}_i^2 = \hat{I}$ it can be rewritten in the following form: 

$$\hat{R}_{\vec{n}} (\alpha) = I \cos{\frac{\alpha}{2}} - i \vec{n} \cdot \hat{\vec{\sigma}} \sin{\frac{\alpha}{2}}$$

Thus the function to compute such operator is:

In [9]:
px = np.array([[0, 1 ], [1 , 0]], dtype = 'complex128')
py = np.array([[0,-1j], [1j, 0]], dtype = 'complex128')
pz = np.array([[1, 0 ], [0 ,-1]], dtype = 'complex128')
pi = np.array([[1, 0 ], [0 , 1]], dtype = 'complex128')

In [10]:
def rot_mat(vec, angle):
    assert len(vec) == 3
    lvec = np.array(vec, dtype = 'float64')
    lvec = lvec / np.linalg.norm(lvec)
    mat = lvec[0] * px + lvec[1] * py + lvec[2] * pz
    sq_mat = mat @ mat
    assert np.max(np.abs(sq_mat - pi)) < 1e-4
    res = pi * np.cos(angle / 2) - 1j * mat * np.sin(angle / 2)
    sq_res = np.conjugate(res).T @ res
    assert np.max(np.abs(sq_res - pi)) < 1e-4
    return res

We needs not general but specific rotation:

In [11]:
rot = rot_mat([1, 1, 1], 2 * np.pi / 3)

In [12]:
print(rot)

[[ 0.5-0.5j -0.5-0.5j]
 [ 0.5-0.5j  0.5+0.5j]]


We need to choose some random (but normalized) initial state:

In [13]:
state = np.random.rand(2) + 1j * np.random.rand(2)
state = state / np.linalg.norm(state)
psi_angle = np.angle(state[0])
state = np.exp(-1j * psi_angle) * state
assert np.imag(state[0]) < 1.e-4

Since here state has form: 
$\vert \psi \rangle = e^{i\psi} \sin \theta \vert 0 \rangle + e^{i\phi} \cos \theta \vert 1 \rangle$
We can rewrite in the following form:
$\vert \psi \rangle = e^{i\psi} (\sin \theta \vert 0 \rangle + \exp{i\varphi} \cos \theta \vert 1 \rangle)$, thus $\varphi = \phi - \psi$.

In [14]:
varphi = np.angle(state[1])
theta  = np.arcsin(state[0])

In [15]:
varphi, theta

(-1.208271575021585, (1.1809914848371965-1.4607893831221532e-16j))

Let's compute symbolic result:

In [16]:
tht, phi = symbols('theta, phi', real = True)

In [17]:
vec0 = Matrix([sin(theta), exp(I * phi) * cos(tht)])

In [18]:
rot_op = Matrix([[1-I, -1-I], [1-I, 1+I]]) / 2

In [19]:
vec1 = rot_op @ vec0

In [20]:
ratio = ((conjugate(vec1[0]) * vec1[0]) / (conjugate(vec1[1]) * vec1[1]))

As we can see ratio of probabilities is:
$$\frac{p(\vert 0 \rangle)}{p(\vert 1 \rangle)} = - \frac{e^{i\phi} (\sin{2\theta} \sin{\phi} + 1)}{e^{i\phi} (\sin{2\theta} \sin{\phi} - 1)} = r$$
Thus the output is:
$$\sin{\phi} \sin{2 \theta} = \frac{r - 1}{r + 1}$$

In [21]:
q = qk.QuantumRegister(1)
c = qk.ClassicalRegister(1)
qc = qk.QuantumCircuit(q, c)
qc.initialize(state, q)
qc.unitary(rot, q)
qc.measure(q, c)
res = qk.execute(qc, backend = simd_backend, shots = 65536).result()
r_simd = res.get_counts()

In [22]:
print(r_simd)

{'0': 11112, '1': 54424}


In [23]:
qc.draw()

┌───────────────────────────────────────┐┌─────────┐┌─┐
q0_0: ┤ initialize(0.92498,0.13476+-0.35531j) ├┤ unitary ├┤M├
      └───────────────────────────────────────┘└─────────┘└╥┘
c0: 1/═════════════════════════════════════════════════════╩═
                                                           0

Let's compute $\sin \phi$:

In [24]:
r = r_simd['0'] / r_simd['1']
sin_phi = (r - 1) / (r + 1) / np.sin(2 * theta)

In [25]:
res_phi = np.arcsin(sin_phi)

In [26]:
res_phi

(-1.22290934272268+8.150324129569399e-16j)

## Task 3

In [27]:
hm = np.array([[1, 1], [1, -1]], dtype = 'complex128') / np.sqrt(2)
sm = np.array([[1, 0], [0, 1j]], dtype = 'complex128')

In [28]:
hadm = Matrix([[1, 1], [1, -1]]) / sqrt(2)

We get three angles $\alpha, \beta, \gamma$. Steps of Euler rotation are:

1. Perform $\hat{R}_z (\gamma)$
2. Perform $\hat{R}_x (\beta )$
3. Perform $\hat{R}_z (\gamma)$

To do so we need to express $\hat{R}_x$ as combination of $\hat{H},\hat{S},\hat{X}$ and $\hat{R}_z$. It can be done in many different ways. But I prefer to decompose it in the following form: $ \hat{R}_x = \hat{H} \hat{R}_z \hat{H}$.


In [29]:
rx = cos(tht / 2) * eye(2) - I * sin(tht / 2) * px
rz = cos(tht / 2) * eye(2) - I * sin(tht / 2) * pz

In [30]:
custom_rx = hadm @ rz @ hadm
rx - custom_rx

Matrix([
[0, 0],
[0, 0]])

Thus we can write our own $\hat{U}_3$ gate:

In [31]:
def custom_rx(qc, qr, alpha):
    qc.h(qr)
    qc.rz(alpha, qr)
    qc.h(qr)

In [32]:
def custom_u3_v0(qc, qr, alpha, beta, gamma):
    qc.rz(beta, qr)
    custom_rx(qc, qr, - np.pi / 2)
    qc.rz(alpha, qr)
    custom_rx(qc, qr,   np.pi / 2)
    qc.rx(gamma, qr)

Let's get Euler's angles from vector:

In [33]:
def rotMatrixFromZX(vz, vx = [1, 0, 0]):
    vz, vx = np.array(vz), np.array(vx)
    nx = vx / np.linalg.norm(vx)
    nz = vz / np.linalg.norm(vz)
    assert(abs(np.dot(nx, nz)) > 1e-10)
    ny = np.cross(nz, nx)
    return np.array([nx, ny, nz])

In [34]:
def _angleFromSinCos(s, c):
    s, c = float(s), float(c)
    if c > 0:
        return np.arctan(s / c)
    elif c < 0:
        return - np.arctan(- s / c)
    else:
        return np.pi * np.sign(s) / 2
    
def rotMatrixToEuler(m):
    cb = m[2, 2]
    if abs(cb) == 1:
        sb = 0
        cc = 1
        sc = 0
        sa = m[1, 0]
        ca = m[0, 0]
    else:
        sb = sqrt(1 - cb**2)
        cc = m[0, 2] / sb
        sc = m[1, 2] / sb
        ca = - m[2, 0] / sb
        sa = m[2, 1] / sb
    return np.array([_angleFromSinCos(sa, ca), _angleFromSinCos(sb, cb), _angleFromSinCos(sc, cc)])

In [35]:
def vecFromAngles(theta_, phi_):
    return np.array([np.sin(theta_) * np.cos(phi_), np.sin(theta_) * np.sin(phi_), np.cos(theta_)])

Let's view on specific vector:

In [36]:
targ_vec = vecFromAngles(np.pi / 3, np.pi / 3)
targ_mat = rotMatrixFromZX(targ_vec)
targ_ans = rotMatrixToEuler(targ_mat)

In [37]:
q = qk.QuantumRegister(1)
c = qk.ClassicalRegister(1)
qc = qk.QuantumCircuit(q, c)
#to custom angles
qc.rz(targ_ans[1],     q[0])
qc.h(q[0])
qc.rz(- np.pi / 2, q[0])
qc.h(q[0])
qc.rz(targ_ans[0],  q[0])
qc.h(q[0])
qc.rz(  np.pi / 2, q[0])
qc.h(q[0])
qc.rz(targ_ans[2],  q[0])
qc.measure(q, c)
#back transformation
res = qk.execute(qc, backend = simd_backend, shots = 65536).result()
r_simd_v0 = res.get_counts()

In [38]:
r_simd_v0

{'0': 49095, '1': 16441}

In [39]:
qc.draw()

┌──────────┐┌───┐┌───────────┐┌───┐┌───────────┐┌───┐┌──────────┐┌───┐»
q4_0: ┤ RZ(pi/3) ├┤ H ├┤ RZ(-pi/2) ├┤ H ├┤ RZ(-pi/3) ├┤ H ├┤ RZ(pi/2) ├┤ H ├»
      └──────────┘└───┘└───────────┘└───┘└───────────┘└───┘└──────────┘└───┘»
c1: 1/══════════════════════════════════════════════════════════════════════»
                                                                            »
«      ┌───────────┐┌─┐
«q4_0: ┤ RZ(-pi/2) ├┤M├
«      └───────────┘└╥┘
«c1: 1/══════════════╩═
«                    0

In [40]:
res_theta = np.arctan(np.sqrt(r_simd_v0['0'] / r_simd_v0['1']))
res_theta, res_theta - np.pi / 3

(1.0461938302562084, -0.0010037209403892522)

In [41]:
q = qk.QuantumRegister(1)
c = qk.ClassicalRegister(1)
qc = qk.QuantumCircuit(q, c)
#to custom angles
qc.rz(targ_ans[1],     q[0])
qc.h(q[0])
qc.rz(- np.pi / 2, q[0])
qc.h(q[0])
qc.rz(targ_ans[0],  q[0])
qc.h(q[0])
qc.rz(  np.pi / 2, q[0])
qc.h(q[0])
qc.rz(targ_ans[2],  q[0])
#as in previous task
rot = rot_mat([1, 1, 1], 2 * np.pi / 3)
qc.unitary(rot, q)
qc.measure(q, c)
res = qk.execute(qc, backend = simd_backend, shots = 65536).result()
r_simd_v0 = res.get_counts()

In [42]:
r_simd_v0

{'0': 4265, '1': 61271}

Let's compare phis using indirect method (by comparing of the measured ratio with the theoretical one)

In [43]:
rat = r_simd_v0['0'] / r_simd_v0['1']
comp_rat = - (np.sin(2 * res_theta) * sin(np.pi / 3) + 1.) / (np.sin(2 * res_theta) * sin(np.pi / 3) - 1.)
ratio_of_ratios = rat / comp_rat
ratio_of_ratios

0.00990468576550399

## Task 4

### Sub-Task 1

In [44]:
hadm = Matrix([[1, 1], [1, -1]]) / sqrt(2)
xmat = Matrix([[0, 1], [1, 0]])

In [45]:
res1 = hadm @ xmat @ hadm

In [46]:
res1

Matrix([
[1,  0],
[0, -1]])

In [47]:
zmat = Matrix([[1, 0], [0, -1]])

In [48]:
res1 - zmat

Matrix([
[0, 0],
[0, 0]])

### Sub-Task 2

Let's find Controlled Z matrix by definition: it should apply $\hat Z$ to first qubit if the second one (control) is in $\vert 1 \rangle_2$ state. That gives us the following transition table:

|Before &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| After &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|--------------------|----------------------------|
|$\vert 00 \rangle$  | $+1 \cdot \vert 00 \rangle$|
|$\vert 10 \rangle$  | $+1 \cdot \vert 10 \rangle$|
|$\vert 01 \rangle$  | $+1 \cdot \vert 01 \rangle$|
|$\vert 11 \rangle$  | $-1 \cdot \vert 11 \rangle$|

Thus it can be represented in form of matrix as:

$$  \begin{bmatrix}
    1 & 0 & 0 & 0\\
    0 & 1 & 0 & 0\\
    0 & 0 & 1 & 0\\
    0 & 0 & 0 &-1
    \end{bmatrix}    $$
    
Or by formula: $\hat{CZ}: \vert ij \rangle \rightarrow (-1)^{i \cdot j} \vert ij \rangle$, obviously it symmetrical for $i,j$ since that $\hat{CZ}$ is equal in both cases.


### Sub-Task 3

First of all we should determine what is Hadamar operator in terms of 2-qubits. Lets's assume that it is applied to the first one, thus transition table is:

|Before &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| After &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|--------------------|-----------------------------------------------------------------|
|$\vert 0x \rangle$  | $\frac{1}{\sqrt{2}} \cdot (\vert 0x \rangle + \vert 1x \rangle)$|
|$\vert 1x \rangle$  | $\frac{1}{\sqrt{2}} \cdot (\vert 0x \rangle - \vert 1x \rangle)$|

Or it can be written as a tensor product:
$\hat{H}^{(2)}_{1} = \hat{I}^{(1)}_{2} \otimes \hat{H}^{(1)}_{1}$. That gives us the followinfg form:

In [49]:
from sympy.physics.quantum import TensorProduct

In [50]:
hadm_2_1 = TensorProduct(eye(2), hadm)

In [51]:
hadm_2_1

Matrix([
[sqrt(2)/2,  sqrt(2)/2,         0,          0],
[sqrt(2)/2, -sqrt(2)/2,         0,          0],
[        0,          0, sqrt(2)/2,  sqrt(2)/2],
[        0,          0, sqrt(2)/2, -sqrt(2)/2]])

And vice-versa for hadamar gate on the second qubit:

In [52]:
hadm_2_2 = TensorProduct(hadm, eye(2))

In [53]:
hadm_2_2

Matrix([
[sqrt(2)/2,         0,  sqrt(2)/2,          0],
[        0, sqrt(2)/2,          0,  sqrt(2)/2],
[sqrt(2)/2,         0, -sqrt(2)/2,          0],
[        0, sqrt(2)/2,          0, -sqrt(2)/2]])

The definition of controlled not for the second gate: it should invert the second qubit if the first one is in $\vert 1 \rangle$ state.

|Before &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| After &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|--------------------|-------------------|
|$\vert 00 \rangle$  | $\vert 00 \rangle$|
|$\vert 10 \rangle$  | $\vert 11 \rangle$|
|$\vert 01 \rangle$  | $\vert 01 \rangle$|
|$\vert 11 \rangle$  | $\vert 10 \rangle$|

That gives us the following form:

In [54]:
cnot_2 = Matrix([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]])

In [55]:
cnot_2

Matrix([
[1, 0, 0, 0],
[0, 0, 0, 1],
[0, 0, 1, 0],
[0, 1, 0, 0]])

Let's writre the right part of circuit:

In [56]:
rc = hadm_2_1 @ hadm_2_2 @ cnot_2 @ hadm_2_1 @ hadm_2_2

In [57]:
rc

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 0, 1],
[0, 0, 1, 0]])

In [58]:
cnot = Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])

In [59]:
rc - cnot

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

### Sub-Task 4

We need to find controlled $e^{i\alpha}$ gate. It should work as: multiply both components 
of the second cubit on $e^{i\alpha}$ if the first one is in $\vert 1 \langle$ state. It's diagram:

|Before &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| After &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|--------------------|--------------------------------------|
|$\vert 00 \rangle$  | $1            \cdot \vert 00 \rangle$|
|$\vert 10 \rangle$  | $e^{i \alpha} \cdot \vert 10 \rangle$|
|$\vert 01 \rangle$  | $1            \cdot \vert 01 \rangle$|
|$\vert 11 \rangle$  | $e^{i \alpha} \cdot \vert 11 \rangle$|

Thus the operator's matrix is the following:

$$  \begin{bmatrix}
    1 & 0 & 0 & 0\\
    0 & e^{i\alpha} & 0 & 0\\
    0 & 0 & 1 & 0\\
    0 & 0 & 0 &e^{i\alpha}
    \end{bmatrix}    $$
    
Obviously it's just another representation of the following tensor product:

$\hat{CExp} = I^{(1)}_{2} \otimes \hat{U}_1 (\alpha)$ and that corresponds just $U_1$ gate on the first qubit (see Sub-Task 3).

## Task 5

If state is represented as $a \vert 0 \rangle \otimes \vert 0 \rangle + b \vert 1 \rangle \otimes \vert 0 \rangle + 
c \vert 0 \rangle \otimes \vert 1 \rangle + \vert 1 \rangle \otimes \vert 1 \rangle$ we can consider that in case of unentanglement it can be written $(\alpha \vert 0 \rangle + \beta \vert 1 \rangle) \otimes (\gamma \vert 0 \rangle + \delta \vert 1 \rangle)$. Thus we can write the following system:

$$
\begin{aligned} 
    \alpha \gamma = a & &\alpha \delta = b \\
    \beta \gamma  = c & &\beta \delta  = d 
\end{aligned}
$$

The criteria of solvable system in this case can be written as: $a d - b c = 0$. In this case this system is unentangled. Also we can solve system to find decomposition.

### Sub-Task 1

Wavefunction is $\frac{2}{3} \vert 0 0 \rangle + \frac{1}{3} \vert 01 \rangle -  \frac{2}{3} \vert 11 \rangle$.

In [60]:
psi = Matrix([[2 / 3, 0], [1 / 3, - 2 / 3]])
psi

Matrix([
[0.666666666666667,                  0],
[0.333333333333333, -0.666666666666667]])

In [61]:
det = psi.det()
print(det)

-0.444444444444444


This state is entangled

In [62]:
alpha, beta, gamma, delta = symbols('alpha, beta, gamma, delta')
mat = Matrix([[alpha * gamma, alpha * delta], [beta * gamma, beta * delta]])
mat

Matrix([
[alpha*gamma, alpha*delta],
[ beta*gamma,  beta*delta]])

In [63]:
eqs = list(mat - psi)
eqs

[alpha*gamma - 0.666666666666667,
 alpha*delta,
 beta*gamma - 0.333333333333333,
 beta*delta + 0.666666666666667]

In [64]:
solve(eqs, alpha, beta, gamma, delta)

[]

This system can not be solved since it's entangled

### Sub-Task 2

Wavefunction is $\frac{1}{2} \vert 0 0 \rangle + \frac{i}{2} \vert 10 \rangle - 
\frac{i}{2} \vert 01 \rangle + \frac{1}{2} \vert 11 \rangle$.

In [65]:
psi = Matrix([[1, I], [-I, 1]]) / 2
psi

Matrix([
[ 1/2, I/2],
[-I/2, 1/2]])

In [66]:
det = psi.det()
print(det)

0


This state is unentangled

In [67]:
eqs = list(mat - psi)
eqs.append(alpha * conjugate(alpha) + beta * conjugate(beta) - 1)
salpha, sbeta, sgamma, sdelta = solve(eqs, alpha, beta, gamma, delta)[0]
salpha, sbeta, sgamma, sdelta

(-sqrt(2)/2, sqrt(2)*I/2, -sqrt(2)/2, -sqrt(2)*I/2)

### Sub-Task 3

Wavefunction is $\frac{1}{2} \vert 0 0 \rangle + \frac{1}{2} \vert 10 \rangle - 
\frac{1}{2} \vert 01 \rangle + \frac{1}{2} \vert 11 \rangle$.

In [68]:
psi = Matrix([[1, 1], [-1, 1]]) / 2
psi

Matrix([
[ 1/2, 1/2],
[-1/2, 1/2]])

In [69]:
det = psi.det()
print(det)

1/2


This state is entangled too

In [70]:
eqs = list(mat - psi)
#eqs.append(alpha * conjugate(alpha) + beta * conjugate(beta) - 1)
sls = solve(eqs, alpha, beta, gamma, delta)
sls

[]

## Task 6

We know that 2-qubit state can be represented as: $\vert \psi \rangle = \sum_{i, j = 0, 1} \psi_{ij} \vert i \rangle \vert j \rangle$. Thus we can exploit Schmidt's decomposition to factorize the righ side of this equality to the following form: $\psi_{ij} = \sum_{k} a_k u_{ik} v_{jk}$. Fortunately this subroutinу is an essential part of numpy.

### Sub-Task 1

We need to set qubits into the state $\vert 00 \rangle + \vert 11 \rangle$ that gives us $\psi_{ij} = \delta_{ij}$.

In [71]:
psis = np.array([[1, 0], [0, -1]])
u, s, v = np.linalg.svd(psis)
a, c = s / np.linalg.norm(s)
b =   c / np.sqrt(a * np.conj(a) + c * np.conj(c))
d = - a / np.sqrt(a * np.conj(a) + c * np.conj(c))
s = np.array([a, c, b, d]).reshape(2, 2)

In [72]:
ut = np.array(TensorProduct(eye(2), Matrix(u))).astype(dtype = 'complex64')
vt = np.array(TensorProduct(Matrix(v), eye(2))).astype(dtype = 'complex64')
st = np.array(TensorProduct(eye(2), Matrix(s))).astype(dtype = 'complex64')

In [73]:
q = qk.QuantumRegister(2)
c = qk.ClassicalRegister(2)
qc = qk.QuantumCircuit(q, c)
qc.unitary(st, q)
qc.cnot(q[0], q[1])
qc.unitary(ut, q)
qc.unitary(vt, q)
qc.measure(q, c)
res = qk.execute(qc, backend = simd_backend, shots = 65536).result()
r_simd = res.get_counts()

In [74]:
qc.draw()

┌──────────┐     ┌──────────┐┌──────────┐┌─┐   
q13_0: ┤0         ├──■──┤0         ├┤0         ├┤M├───
       │  unitary │┌─┴─┐│  unitary ││  unitary │└╥┘┌─┐
q13_1: ┤1         ├┤ X ├┤1         ├┤1         ├─╫─┤M├
       └──────────┘└───┘└──────────┘└──────────┘ ║ └╥┘
 c3: 2/══════════════════════════════════════════╩══╩═
                                                 0  1

In [75]:
r_simd

{'00': 32765, '11': 32771}

### Sub-Task 2

In [76]:
psis = np.array([[0, 1], [1, 0]])
u, s, v = np.linalg.svd(psis)
a, c = s / np.linalg.norm(s)
b =   c / np.sqrt(a * np.conj(a) + c * np.conj(c))
d = - a / np.sqrt(a * np.conj(a) + c * np.conj(c))
s = np.array([a, c, b, d]).reshape(2, 2)

In [77]:
ut = np.array(TensorProduct(eye(2), Matrix(u))).astype(dtype = 'complex64')
vt = np.array(TensorProduct(Matrix(v), eye(2))).astype(dtype = 'complex64')
st = np.array(TensorProduct(eye(2), Matrix(s))).astype(dtype = 'complex64')

In [78]:
q = qk.QuantumRegister(2)
c = qk.ClassicalRegister(2)
qc = qk.QuantumCircuit(q, c)
qc.unitary(st, q)
qc.cnot(q[0], q[1])
qc.unitary(ut, q)
qc.unitary(vt, q)
qc.measure(q, c)
res = qk.execute(qc, backend = simd_backend, shots = 65536).result()
r_simd = res.get_counts()

In [79]:
r_simd

{'01': 32709, '10': 32827}

### Sub-Task 3

In [80]:
psis = np.array([[3, 1], [1, -1]])
u, s, v = np.linalg.svd(psis)
print(s)
a, c = s / np.linalg.norm(s)
b =   c / np.sqrt(a * np.conj(a) + c * np.conj(c))
d = - a / np.sqrt(a * np.conj(a) + c * np.conj(c))
s = np.array([a, c, b, d]).reshape(2, 2)

[3.23606798 1.23606798]


In [81]:
ut = np.array(TensorProduct(eye(2), Matrix(u))).astype(dtype = 'complex64')
vt = np.array(TensorProduct(Matrix(v), eye(2))).astype(dtype = 'complex64')
st = np.array(TensorProduct(eye(2), Matrix(s))).astype(dtype = 'complex64')

In [82]:
full_mat = st @ cnot_2 @ ut @ vt.T
full_mat @ np.array([1, 0, 0, 0])

array([0.903696140572831, 0.288675171919875, 0.288675130830658,
       -0.129099435527793], dtype=object)

In [83]:
q = qk.QuantumRegister(2)
c = qk.ClassicalRegister(2)
qc = qk.QuantumCircuit(q, c)
qc.unitary(st, q)
qc.cnot(q[0], q[1])
qc.unitary(ut, q)
qc.unitary(vt.T, q)
qc.measure(q, c)
res = qk.execute(qc, backend = simd_backend, shots = 65536).result()
r_simd = res.get_counts()

In [84]:
r_simd

{'00': 49331, '01': 5404, '10': 5392, '11': 5409}